# Running a multi-member hydrological ensemble on the Raven Server

Here we use birdy's WPS client to launch the GR4JCN and HMETS hydrological models on the server and analyze the outputs. 

In [1]:
from birdy import WPSClient

from example_data import TESTDATA
import datetime as dt
from urllib.request import urlretrieve
import xarray as xr
import numpy as np
from matplotlib import pyplot as plt

url = "http://localhost:9099/wps"
wps = WPSClient(url)

In [17]:
# The model parameters for gr4jcn and hmets. Can either be a string of comma separated values, a list, an array or a named tuple. 
gr4jcn1 ='0.529, -3.396, 407.29, 1.072, 16.9, 0.947'
gr4jcn2 ='0.4, -3.96, 307.29, 1.072, 16.9, 0.947'

params = [gr4jcn1,gr4jcn2] # Rich Test 1



# Forcing files. Raven uses the same forcing files for all and extracts the information it requires for each model.
ts=TESTDATA['raven-gr4j-cemaneige-nc-ts']

# Model configuration parameters.
config = dict(
    start_date=dt.datetime(2000, 1, 1),
    end_date=dt.datetime(2002, 1, 1),
    area=4250.6,
    elevation=843.0,
    latitude=54.4848,
    longitude=-123.3659,
    )

# Launch the WPS to get the multi-model results.  Note the "gr4jcn" and "hmets" keys.
resp=wps.raven_gr4j_cemaneige(ts=str(ts),params=params, **config) 




In [18]:
# And get the response
# With `asobj` set to False, only the reference to the output is returned in the response. 
# Setting `asobj` to True does nt work here as the outputs are zipped files and could be numerous.
# The user can download the resulting data. So we use asobj=False
[hydrograph, storage, solution, diagnostics] = resp.get(asobj=False)


# Here the code crashes on my instance with Errno:
[Errno -51] NetCDF: Unknown file format: b'/tmp/tmp77v34gl4'

Perhaps the NetCDF reader is not working well? Xarray should be installed. Need to verify please.

In [22]:
print(hydrograph)
print(diagnostics)

http://localhost:9099/outputs/8323a802-9448-11e9-8b59-b8ca3a8f5177/Hydrographs.nc
http://localhost:9099/outputs/8323a802-9448-11e9-8b59-b8ca3a8f5177/Diagnostics.zip


The `hydrograph` and `diagnostics` files can be downloaded by clicking the above links. hydrographs is a netcdf file containing the multiple simulations, and diagnostics is a zip-file containing the various diagnostics files linked to each parameter set.